In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
npv = [5.6, 2.7, 3.9, 6.8, 7.7, 5.1, 2.8, 3.5, 4.5, 5.1, 6.1, 9.5, 7.2, 4.9, 3.1, 3.6, 1.55, 0.29, 5.2, 5.9, 6.1, 3.5, 4.5, 5.2, 2.1, 3.7]
len(npv)

In [ ]:
#Capital Budgeting Base Problem Data
cost = [5, 2.4, 3.5, 5.9, 6.9, 4.4, 2.5, 3.1, 3.2, 4, 5.1, 6.1, 4.4, 4.4, 2.5, 3.1, 2.15, 1.57, 2.3, 5.4, 5, 1.9, 2.5, 4, 1.5, 3]
npv = [5.6, 2.7, 3.9, 6.8, 7.7, 5.1, 2.8, 3.5, 4.5, 5.1, 6.1, 9.5, 7.2, 4.9, 3.1, 3.6, 1.55, 0.29, 5.2, 5.9, 6.1, 3.5, 4.5, 5.2, 2.1, 3.7]
budget = 50
n = len(cost)

#define the concrete model
model = ConcreteModel()

#DVs
model.x = Var(range(n), domain = Binary) #NOTE: This is where we restrict the decision variables to be binary (could also put things like NonNegativeIntegers)

#budget constraint
model.BudgetConstraint = Constraint(expr = sum(cost[i]*model.x[i] for i in range(n)) <= budget)

#objective
model.Objective = Objective(expr = sum(npv[i]*model.x[i] for i in range(n)), sense = maximize)

In [ ]:
#problem 1 - constraint 1
#you can add at most two investments from 20 to 26
#constraint: at most 2 investments from numbers 20-26
#model.InvestmentLimit=Constraint(expr=sum(model.x[i] for i in range (19,26))<=2)
#model.pprint()

In [ ]:
#problem 2- constraint 2
#if you invest in number 11, you must also invest in 12
#model.InvestmentLimit=Constraint(expr=model.x[10]<=model.x[11])


In [ ]:
#problem 3
#must invest in 6 to be able to invest in any of 7-10
model.InvestmentLimit = ConstraintList()
for i in range(6, 10):
    model.InvestmentLimit.add(expr=model.x[i]<= model.x[5])

In [ ]:
#problem 4 guarantee global optimality using by setting ratio gap to 0
#solve the model
opt = SolverFactory('cbc')
opt.options['seconds'] = 5 #specifies the time limit (in seconds)
opt.options['ratioGap'] = 0 #specifies the optimality gap tolerance (.01 means alg can stop if guarenteed within <1% of optimal obj)
results = opt.solve(model, tee=True)

In [ ]:
#print out solution
print("invest in options: ")
for i in range(n):
    if model.x[i]() == 1:
        print(i)

print("Total NPV:", model.Objective())
print("Total Cost:", model.BudgetConstraint())

In [ ]:
#must invest in 6 to be able to invest in any of 7-10
#model.InvestmentLimit = ConstraintList()
#for i in range(6, 10):
 #   model.InvestmentLimit.add(expr = model.x[i] <= model.x[5])

#solve and check
#opt = SolverFactory('cbc')
#results = opt.solve(model, tee=True)

#print which investments were selected
#print("\nSelected investments:")
#for i in range(n):
 #   if model.x[i].value == 1:
  #      print(f"Investment {i+1} (index {i})")